In [181]:
import pandas as pd
import numpy as np
import datasets
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate

from transformers import Trainer
from sklearn.metrics import classification_report
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
train_data_path = "./data/English dataset/train.jsonl"
test_data_path = "./data/English dataset/test.jsonl"

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Domen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Domen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Domen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Domen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [182]:
def pretty_print_report_dict(report):
	report_df = pd.DataFrame(report).transpose()
	report_df = report_df.round(3)

	class_metrics = report_df.iloc[:-3, :].copy()

	summary_metrics = report_df.iloc[-3:, :].copy()
	summary_metrics = summary_metrics.drop(columns=['support'])

	print("CLASS PERFORMANCE")
	print(tabulate(class_metrics, headers='keys', tablefmt='heavy_outline', numalign="center"))
	print()
	print("GLOBAL AVERAGES")
	print(tabulate(summary_metrics, headers='keys', tablefmt='heavy_outline', numalign="center"))

# Data preprocessing

In [183]:
def preprocess_text(text): # From the labs
	# Tokenize the text into words
	words = word_tokenize(text.lower())  # Convert text to lowercase

	# Remove punctuation
	table = str.maketrans('', '', string.punctuation)
	words = [word.translate(table) for word in words if word.isalpha()]

	# Remove stopwords
	stop_words = set(stopwords.words('english'))
	words = [word for word in words if word not in stop_words]

	# Lemmatization
	lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

	# Join the words back into a string
	preprocessed_text = ' '.join(lemmatized_words)
	return preprocessed_text

(We load the dataset. We join togeder eintailment and not mentioned, so we can focus on predicting only if something is a contradiction)

In [184]:
train_data = pd.DataFrame(datasets.load_dataset("json", data_files=train_data_path)["train"])
test_data = pd.DataFrame(datasets.load_dataset("json", data_files=test_data_path)["train"])

label_map = {"Contradiction": 1, "Entailment": 0, "NotMentioned": 0}
train_data["label"] = train_data["label"].map(label_map)
test_data["label"] = test_data["label"].map(label_map)

train_data = train_data.drop("doc_id", axis=1)
train_data = train_data.drop("key", axis=1)
test_data = test_data.drop("doc_id", axis=1)
test_data = test_data.drop("key", axis=1)

(After we load the dataset, we inspect it for class inbalance)

In [185]:
train_data["label"].value_counts(normalize=True)

label
0    0.883048
1    0.116952
Name: proportion, dtype: float64

(We can see that most of the data isn't contradictions. The data is quite imbalanced)

In [186]:
longest_premise = max(train_data['premise'].apply(len).max(), test_data['premise'].apply(len).max())
longest_hypotises = max(train_data['hypothesis'].apply(len).max(), test_data['hypothesis'].apply(len).max())
longest_sentance = max(longest_premise, longest_hypotises)
print("Longest premise: ", longest_premise)
print("Longest hypothesis: ", longest_hypotises)

print("---------------------------------")
mean = np.mean(train_data['premise'].apply(len))
std = np.std(train_data['premise'].apply(len))

print("Mean premise length: ", mean)
print("+1 std: ", mean+std)
print("+2 std: ", mean+2*std)
print("+3 std: ", mean+3*std)

Longest premise:  3098
Longest hypothesis:  162
---------------------------------
Mean premise length:  296.27826449728826
+1 std:  651.2505192635402
+2 std:  1006.2227740297922
+3 std:  1361.1950287960442


(We inspect the lenght of the data. We do this to see if it would be beneficial only keeping smaller sizes of the data, so we can cleanly feed it into BERT model. We conclude that we would need to thin our data too much to be worth it)

# Traditional ML

In [187]:
train_data_preprocessed = train_data.copy()
test_data_preprocessed = test_data.copy()

train_data_preprocessed["premise"] = train_data_preprocessed["premise"].map(preprocess_text)
train_data_preprocessed["hypothesis"] = train_data_preprocessed["hypothesis"].map(preprocess_text)

test_data_preprocessed["premise"] = test_data_preprocessed["premise"].map(preprocess_text)
test_data_preprocessed["hypothesis"] = test_data_preprocessed["hypothesis"].map(preprocess_text)

vectorizer_premise = TfidfVectorizer()
vectorizer_hypothesis = TfidfVectorizer()

train_data_vectorised = train_data.copy()

X_premise= vectorizer_premise.fit_transform(train_data["premise"])
X_hypothesis = vectorizer_hypothesis.fit_transform(train_data["hypothesis"])
train_data_vectorised = hstack([X_premise, X_hypothesis])

Y_premise = vectorizer_premise.transform(test_data_preprocessed["premise"])
Y_hypothesis = vectorizer_hypothesis.transform(test_data_preprocessed["hypothesis"])
test_data_vectorised = hstack([Y_premise, Y_hypothesis])

## Logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

grid_serch_dict = {
	"l1_ratio": [0, 0.5, 1],
	"C": [0.1, 0.5, 1.0, 2.0, 10, 50],
	"class_weight": [None, "balanced"],
	"solver": ["saga"]
}

logreg_model = RandomizedSearchCV(LogisticRegression(max_iter=5000), grid_serch_dict, n_iter=10, cv=3, scoring='f1')
logreg_model.fit(train_data_vectorised, train_data["label"])

predictions = logreg_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in logreg_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

c:\Users\timna\Desktop\SkupnoAng\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Hyperparameter   ┃ Value    ┃
┣━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━┫
┃ solver           ┃ saga     ┃
┃ l1_ratio         ┃ 1        ┃
┃ class_weight     ┃ balanced ┃
┃ C                ┃ 50       ┃
┗━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━┛


In [9]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.926    ┃  0.997   ┃    0.96    ┃   1871    ┃
┃ 1  ┃    0.923    ┃  0.327   ┃   0.483    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.926    ┃  0.926   ┃   0.926    ┃
┃ macro avg    ┃    0.925    ┃  0.662   ┃   0.722    ┃
┃ weighted avg ┃    0.926    ┃  0.926   ┃    0.91    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


## Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

grid_serch_dict = {
	"n_estimators": [50, 100, 200],
	"max_depth": [None, 5, 10, 20],
	"min_samples_split": [2, 5, 10, 20, 50],
	"class_weight": [None, "balanced"]
}

rf_model = RandomizedSearchCV(RandomForestClassifier(random_state=67), grid_serch_dict, n_iter=15, cv=3, scoring='f1')
rf_model.fit(train_data_vectorised, train_data["label"])

predictions = rf_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in rf_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Hyperparameter    ┃ Value   ┃
┣━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ n_estimators      ┃ 200     ┃
┃ min_samples_split ┃ 5       ┃
┃ max_depth         ┃ None    ┃
┃ class_weight      ┃ None    ┃
┗━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━┛


In [11]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.923    ┃    1     ┃    0.96    ┃   1871    ┃
┃ 1  ┃      1      ┃  0.295   ┃   0.456    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.926    ┃  0.926   ┃   0.926    ┃
┃ macro avg    ┃    0.962    ┃  0.648   ┃   0.708    ┃
┃ weighted avg ┃    0.932    ┃  0.926   ┃   0.907    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


## SVC

In [188]:
from sklearn.svm import SVC

grid_serch_dict = {
	"C": [0.1, 0.5, 1.0, 2.0, 10, 50],
	"kernel": ["linear", "sigmoid", "rbf"],
	"class_weight": [None, "balanced"]
}

svm_model = RandomizedSearchCV(SVC(), grid_serch_dict, n_iter=10, cv=3, scoring='f1')
svm_model.fit(train_data_vectorised, train_data["label"])

predictions = svm_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in svm_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Hyperparameter   ┃ Value   ┃
┣━━━━━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ kernel           ┃ rbf     ┃
┃ class_weight     ┃ None    ┃
┃ C                ┃ 10      ┃
┗━━━━━━━━━━━━━━━━━━┻━━━━━━━━━┛


In [189]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.946    ┃  0.997   ┃   0.971    ┃   1871    ┃
┃ 1  ┃    0.95     ┃  0.518   ┃   0.671    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.946    ┃  0.946   ┃   0.946    ┃
┃ macro avg    ┃    0.948    ┃  0.757   ┃   0.821    ┃
┃ weighted avg ┃    0.947    ┃  0.946   ┃   0.939    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


# Transformer-Based Classifier

## Training

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

def preprocess_function(examples):
	inputs = tokenizer(examples["premise"], examples["hypothesis"], 
			max_length=1024, truncation="only_first", padding="max_length")

	global_attention_mask = [[0] * len(ids) for ids in inputs["input_ids"]]

	for mask in global_attention_mask:
		mask[0] = 1 
		
	inputs["global_attention_mask"] = global_attention_mask
	return inputs

model_name = "kiddothe2b/longformer-mini-1024"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at kiddothe2b/longformer-mini-1024 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)

dataset_split = tokenized_train_dataset.train_test_split(test_size=0.1)
tokenized_train_dataset = dataset_split["train"]
tokenized_eval_dataset = dataset_split["test"]

test_dataset = Dataset.from_pandas(test_data)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 2091/2091 [00:02<00:00, 814.66 examples/s]


In [ ]:
RUN = False

from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score
from torch import nn

class WeightedTrainer(Trainer):
	def __init__(self, *args, class_weights=None, **kwargs):
		super().__init__(*args, **kwargs)
		if class_weights is not None:
			self.class_weights = torch.tensor(class_weights, dtype=torch.float).to(self.args.device)
		else:
			self.class_weights = None

	def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
		labels = inputs.get("labels")
		outputs = model(**inputs)
		logits = outputs.get("logits")
		
		if self.class_weights is not None:
			loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
			loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
		else:
			# Fallback to default loss if no weights are provided
			loss = outputs.loss if isinstance(outputs, dict) else outputs[0]
			
		return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
	logits, labels = eval_pred
	predictions = np.argmax(logits, axis=-1)

	f1 = f1_score(labels, predictions, pos_label=1, average='binary')
	return {"f1_score_class_1": f1}

class_weights = [1.0, 9.0] # Weight class 0, weight class 1

training_args = TrainingArguments(
	output_dir="./artifacts",
	learning_rate=2e-4,
	per_device_train_batch_size=2,
	gradient_accumulation_steps=16,
	num_train_epochs=2,
	weight_decay=0.01,
	save_strategy="steps",
	save_steps=50,
	save_total_limit=3, 
	load_best_model_at_end=True,
	metric_for_best_model="f1_score_class_1", 
	eval_strategy="steps",
	eval_steps=50,
	greater_is_better=True,
	resume_from_checkpoint=False
)


path = "./trained_model_ex3_f1_class1_weighted"
if (not os.path.exists(path) and RUN):
	trainer = WeightedTrainer(
		model=model,
		args=training_args,
		train_dataset=tokenized_train_dataset,
		eval_dataset=tokenized_eval_dataset,
		compute_metrics=compute_metrics,
		class_weights=class_weights,
	)

	trainer.train(resume_from_checkpoint=True)

	tokenizer.save_pretrained(path)
	trainer.save_model(path)
else:
	print("Model not trained")

Model not trained


## Evaluation

In [ ]:
path = "trained_model_ex3_precision_class1_v1"
if (os.path.exists(path)):
	model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

	trainer = Trainer(model=model)
	predictions_procentages = trainer.predict(tokenized_test_dataset)[0]
	predictions = predictions_procentages.argmax(-1)
	report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
	pretty_print_report_dict(report_dict)
else:
	print("Model not found")

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.981    ┃  0.972   ┃   0.976    ┃   1871    ┃
┃ 1  ┃    0.776    ┃  0.836   ┃   0.805    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.957    ┃  0.957   ┃   0.957    ┃
┃ macro avg    ┃    0.878    ┃  0.904   ┃   0.891    ┃
┃ weighted avg ┃    0.959    ┃  0.957   ┃   0.958    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


In [ ]:
path = "trained_model_ex3_v2_macro_f1_v1"
if (os.path.exists(path)):
	model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

	trainer = Trainer(model=model)
	predictions_procentages = trainer.predict(tokenized_test_dataset)[0]
	predictions = predictions_procentages.argmax(-1)
	report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
	pretty_print_report_dict(report_dict)
else:
	print("Model not found")

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.935    ┃  0.975   ┃   0.955    ┃   1871    ┃
┃ 1  ┃    0.667    ┃  0.427   ┃   0.521    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.917    ┃  0.917   ┃   0.917    ┃
┃ macro avg    ┃    0.801    ┃  0.701   ┃   0.738    ┃
┃ weighted avg ┃    0.907    ┃  0.917   ┃   0.909    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


In [ ]:
path = "trained_model_ex3_f1_class1_weighted"
if (os.path.exists(path)):
	model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

	trainer = Trainer(model=model)
	predictions_procentages = trainer.predict(tokenized_test_dataset)[0]
	predictions = predictions_procentages.argmax(-1)
	report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
	pretty_print_report_dict(report_dict)
else:
	print("Model not found")

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.981    ┃  0.975   ┃   0.978    ┃   1871    ┃
┃ 1  ┃    0.797    ┃  0.836   ┃   0.816    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.96     ┃   0.96   ┃    0.96    ┃
┃ macro avg    ┃    0.889    ┃  0.906   ┃   0.897    ┃
┃ weighted avg ┃    0.961    ┃   0.96   ┃   0.961    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


In [ ]:
path = "trained_model_ex3_f1_class1_weighted_2_epoc"
if (os.path.exists(path)):
	model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

	trainer = Trainer(model=model)
	predictions_procentages = trainer.predict(tokenized_test_dataset)[0]
	predictions = predictions_procentages.argmax(-1)
	report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
	pretty_print_report_dict(report_dict)
else:
	print("Model not found")

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.982    ┃  0.974   ┃   0.978    ┃   1871    ┃
┃ 1  ┃    0.791    ┃  0.845   ┃   0.818    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.96     ┃   0.96   ┃    0.96    ┃
┃ macro avg    ┃    0.887    ┃   0.91   ┃   0.898    ┃
┃ weighted avg ┃    0.962    ┃   0.96   ┃   0.961    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


# Task 4

In [57]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import ContrastiveLoss, MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

# 3. Define the Hyperparameter Search Space
def search_space(trial):
    return {
        'learning_rate': trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        'weight_decay': trial.suggest_float("weight_decay", 0.01, 0.1),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.05, 0.2),
        'max_grad_norm': trial.suggest_float("max_grad_norm", 0.5, 1.0, log=True),
        'num_train_epochs': trial.suggest_int("num_train_epochs", 1, 10), # ! lower when using a slower model 
    }

# 4. Define the Model Initialization
def model_init(trial):
    return SentenceTransformer("models\jina-embeddings-v2-small-en", device="cuda" )

# 5. Define the Loss Initialization
def cl_loss_init(model):
    return ContrastiveLoss(model)
def mnlr_loss_init(model):
    return MultipleNegativesRankingLoss(model)

# 6. Define the Objective Function
def hpo_compute_objective(metrics):
    print(metrics)
    return metrics["eval_sts-dev_cosine_recall@10"]

In [42]:
from datasets import Dataset, load_dataset
import pandas as pd

def get_data(valid_split=0.1):
    train_data_path = "./data/English dataset/train.jsonl"
    test_data_path = "./data/English dataset/test.jsonl"

    train_data = pd.DataFrame(load_dataset("json", data_files=train_data_path)["train"])
    test_dataset = pd.DataFrame(load_dataset("json", data_files=test_data_path)["train"])

    label_map = {"Contradiction": 1, "Entailment": 0, "NotMentioned": 0}
    train_data["label"] = train_data["label"].map(label_map)
    test_dataset["label"] = test_dataset["label"].map(label_map)

    train_data = train_data.drop("doc_id", axis=1)
    train_data = train_data.drop("key", axis=1)
    test_dataset = test_dataset.drop("doc_id", axis=1)
    test_dataset = test_dataset.drop("key", axis=1)

    train_data["label"].value_counts(normalize=True)

    ds = Dataset.from_pandas(train_data)
    ds = ds.select_columns(["hypothesis", "premise", "label"])

    dss = ds.train_test_split(valid_split, seed=42)
    train_dataset = dss['train']
    valid_dataset = dss['test']
    test_dataset = Dataset.from_pandas(test_dataset)
    test_dataset = test_dataset.select_columns(["hypothesis", "premise", "label"])

    return train_dataset, valid_dataset, test_dataset

In [158]:
train_dataset, valid_dataset, test_dataset = get_data(valid_split=0.15)

In [10]:
def filter_neg(data, model, params):
    d = {'hypothesis':[], 'premise':[]}
    for k in range(len(data)):
        if data['label'][k] == 1:
            d['hypothesis'].append(data['hypothesis'][k])
            d['premise'].append(data['premise'][k])
    meow = Dataset.from_dict(d)
    return (meow, None)

In [156]:
import random
from datasets import Dataset
from datasets import Dataset
from sentence_transformers.data_collator import SentenceTransformerDataCollator
from torch._tensor import Tensor
from typing import Any

class MyDataCollator(SentenceTransformerDataCollator): # We need this so we can insert negatives to the batches
    universal_negatives = ['', '']
    negatives_per_batch = 4

    def __call__(self, features: list[dict[str, Any]]) -> dict[str, Tensor]:
        k = list(features[0].keys())[1]
        for i, feature in enumerate(features):
            features[i][k] = [features[i][k]] + random.sample(
            self.universal_negatives, 
            min(self.negatives_per_batch, len(self.universal_negatives))
        )
        #print(features[i][k])
        batch = super().__call__(features)
        
        return batch

def get_data_col(data, model, params):
    h = len(set(data['hypothesis']))
    p = len(set(data['premise']))
    
    hmm = dict(zip(set(data['premise']), range(p)))
    d = {'hypothesis':[], 'premise':[]}
    for k in range(len(data)):
        if data['label'][k] == 1:
            d['hypothesis'].append(data['hypothesis'][k])
            d['premise'].append(data['premise'][k])
        else:
            hmm[data['premise'][k]] += 1
    universal_negatives = []
    for k, v in hmm.items():
        if v >= h: 
            universal_negatives.append(k) # if this premise is negative for all hypothesis
    
    data = Dataset.from_dict(d)
    data_collator = MyDataCollator(model.tokenize)
    data_collator.universal_negatives = universal_negatives
    data_collator.negatives_per_batch = params['negatives_per_batch']

    return data, data_collator


In [60]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/checkpoints",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1.2247359733257542e-05,
    seed=42,
    metric_for_best_model=f"eval_cosine_recall@10",
    load_best_model_at_end=True,
    weight_decay=0.09092585204374326,
    warmup_ratio=0.05503071687326718,
    batch_sampler=None,
    #batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=50,
    torch_empty_cache_steps = None,
    save_steps=50,
    save_total_limit=2,
    max_grad_norm= 0.8774817671930895,
    logging_steps=100,
    
)

In [172]:
from sentence_transformers.training_args import BatchSamplers
from copy import deepcopy

args_mnlr = deepcopy(args)
args_mnlr.num_train_epochs = 10
args_mnlr.batch_sampler = BatchSamplers.NO_DUPLICATES
args_mnlr2 = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/checkpoints",
    # Optional training parameters:
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=4.890585631921125e-05,
    seed=42,
    metric_for_best_model=f"eval_cosine_recall@10",
    #load_best_model_at_end=True,
    weight_decay=0.010800797401617856,
    warmup_ratio=0.0765080638605733,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="no",
    eval_steps=100,
    torch_empty_cache_steps = None,
    max_grad_norm= 0.5013390304609416,
    logging_steps=100,
)

MNLR assumes that in each batch every hypothesis has only one correct premise. It treates every other premise as a wrong answer, and tries to distance the hypothesis from it. This isn't true in general, so we need to ensure that during training, there aren't many overlaping premises. It's still possible that multiple correct premises for a hypothesis make it in the batch but NO_DUPLICATES tries to at least minimize that. 

In [173]:
from sentence_transformers.losses import ContrastiveLoss, MultipleNegativesRankingLoss
# Kinda bad, and slow but simple
model_cl_kwargs = {"trained_model_path":"models/trained/model_cl", "loss": cl_loss_init, "args":args, "model_init":model_init}

# kinda bad, but fast, and simple
model_mnlr_kwargs = {
    "trained_model_path":"models/trained/model_mnlr", 
    "loss":mnlr_loss_init, 
    "args":args_mnlr, 
    "data_preprocesser": filter_neg
    }

# our SOTA model
model_mnlr2_kwargs = {
    "trained_model_path":"models/trained/model_mnlr2_test", 
    "loss":mnlr_loss_init, 
    "args":args_mnlr2, 
    "data_preprocesser": get_data_col, 
    "params": {"negatives_per_batch":3},
    "model_init":model_init
    }

In [82]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from collections import defaultdict

def get_ret_eval(test_dataset, name=''):
    corpus = dict(zip(test_dataset['premise'], test_dataset['premise']))
    queries = dict(zip(test_dataset['hypothesis'], test_dataset['hypothesis']))
    relevant_docs = defaultdict(list)

    for k in range(len(test_dataset)):
        if test_dataset['label'][k] > 0:
            relevant_docs[test_dataset['hypothesis'][k]].append(test_dataset['premise'][k])
    

    inf_ret_ev = InformationRetrievalEvaluator(
        queries= queries,
        corpus = corpus,
        relevant_docs = relevant_docs,
        #similarity_fn_names= ["cosine"],
        show_progress_bar=True,
        batch_size= 16,
        #main_score_function="Recall@10",
        name=name
    )

    return inf_ret_ev

In [165]:
from sentence_transformers.losses import ContrastiveLoss
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers import SentenceTransformer
from transformers import EarlyStoppingCallback


def get_trainer(
        train_dataset, 
        valid_dataset, 
        model_init=model_init,
        loss=mnlr_loss_init, 
        args=None, 
        evaluator=None, 
        data_preprocesser=None, 
        params=None,
        trained_model_path=None,
        early_stopping=False
    ):
    #base_model = SentenceTransformer(model_name)
    
    #loss = loss(base_model)
    data_collator = None
    if early_stopping:
        early_stopper = [EarlyStoppingCallback(
            early_stopping_patience=5,
            early_stopping_threshold=0.05
        )]
    else:
        early_stopper = None

    if data_preprocesser is not None:
        train_dataset, data_collator = data_preprocesser(train_dataset, model_init(None), params)

    if valid_dataset is not None and evaluator is None:
        evaluator = get_ret_eval(valid_dataset)

    trainer = SentenceTransformerTrainer(
        model_init = model_init,
        train_dataset = train_dataset,
        eval_dataset = valid_dataset,
        loss = loss,
        evaluator = evaluator,
        args = args,
        data_collator = data_collator,
        callbacks=early_stopper
    )

    return trainer

In [52]:
def hyper_search(train_dataset, valid_dataset, model_kwargs):
    dev_evaluator = get_ret_eval(valid_dataset, name="sts-dev")
    model_kwargs = deepcopy(model_kwargs)
    # 7. Define the training arguments
    hpo_args = SentenceTransformerTrainingArguments(
        # Required parameter:
        num_train_epochs=1.0,
        per_device_train_batch_size=16,
        seed=42,
        metric_for_best_model=f"eval_cosine_recall@10",
        output_dir="checkpoints",
        batch_sampler=BatchSamplers.NO_DUPLICATES, # Remove if searching for the Contrastive loss or triplet model
        # Optional tracking/debugging parameters:
        eval_strategy="no", # We don't need to evaluate/save during HPO
        save_strategy="no",
        logging_steps=40,
        run_name="hpo",  # Will be used in W&B if `wandb` is installed
        
    )
    model_kwargs['args'] = hpo_args
    trainer = get_trainer(train_dataset, valid_dataset, evaluator=dev_evaluator, **model_kwargs)

    best_trial = trainer.hyperparameter_search(
        hp_space=search_space,
        compute_objective=hpo_compute_objective,
        n_trials=25,
        direction="maximize",
        backend="optuna",

    )
    print(best_trial)

In [175]:
def train(train_dataset, valid_dataset, model_kwargs):
    trainer = get_trainer(train_dataset, valid_dataset,early_stopping=False, **model_kwargs)
    trainer.train()
    trainer.model.save_pretrained(model_kwargs['trained_model_path'])

In [112]:
TRAIN = True
HYPER_PARAMETER_SEARCH = False

In [176]:
if TRAIN:
    train(train_dataset, valid_dataset, model_mnlr2_kwargs)

Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Step,Training Loss
100,1.763700
200,1.162300
300,0.567900
400,0.382600
500,0.405700
600,0.338800
700,0.321600
800,0.274100
900,0.218300
1000,0.147700


In [58]:
if HYPER_PARAMETER_SEARCH:
    hyper_search(train_dataset, valid_dataset, model_mnlr2_kwargs)
    

Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

[I 2026-01-04 12:00:36,693] A new study created in memory with name: no-name-bfa4578b-9222-4053-aa49-01dc690d65bf
Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', '

Step,Training Loss
40,1.878400


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:06<00:00,  6.16s/it]


{'eval_loss': 0.41465243697166443, 'eval_sts-dev_cosine_accuracy@1': 0.2222222222222222, 'eval_sts-dev_cosine_accuracy@3': 0.2222222222222222, 'eval_sts-dev_cosine_accuracy@5': 0.5555555555555556, 'eval_sts-dev_cosine_accuracy@10': 0.5555555555555556, 'eval_sts-dev_cosine_precision@1': 0.2222222222222222, 'eval_sts-dev_cosine_precision@3': 0.14814814814814814, 'eval_sts-dev_cosine_precision@5': 0.2222222222222222, 'eval_sts-dev_cosine_precision@10': 0.2, 'eval_sts-dev_cosine_recall@1': 0.012051734273956496, 'eval_sts-dev_cosine_recall@3': 0.020282186948853614, 'eval_sts-dev_cosine_recall@5': 0.17832948944060054, 'eval_sts-dev_cosine_recall@10': 0.22836792836792835, 'eval_sts-dev_cosine_ndcg@10': 0.23898477030516066, 'eval_sts-dev_cosine_mrr@10': 0.30000000000000004, 'eval_sts-dev_cosine_map@100': 0.1512150030499169, 'eval_runtime': 35.8248, 'eval_samples_per_second': 20.098, 'eval_steps_per_second': 2.512}


[I 2026-01-04 14:12:27,142] Trial 24 finished with value: 0.22836792836792835 and parameters: {'learning_rate': 2.653783078613784e-05, 'weight_decay': 0.041641233385403445, 'warmup_ratio': 0.06315440369727221, 'max_grad_norm': 0.5784105499771178, 'num_train_epochs': 6}. Best is trial 15 with value: 0.27251842807398363.


BestRun(run_id='15', objective=0.27251842807398363, hyperparameters={'learning_rate': 4.890585631921125e-05, 'weight_decay': 0.010800797401617856, 'warmup_ratio': 0.0765080638605733, 'max_grad_norm': 0.5013390304609416, 'num_train_epochs': 7}, run_summary=None)


In [ ]:
if TRAIN:
    train(train_dataset, valid_dataset, model_cl_kwargs)

In [ ]:
if TRAIN:
    train(train_dataset, valid_dataset, model_mnlr_kwargs)

In [70]:
if TRAIN:
    train(train_dataset, valid_dataset, model_mnlr2_kwargs)

Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Some weights of BertModel were not initialized from the model checkpoint at models\jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,0.346475,0.222222,0.333333,0.333333,0.444444,0.222222,0.148148,0.133333,0.111111,0.012052,0.024714,0.032944,0.071334,0.127283,0.271605,0.073633
100,1.627300,0.363814,0.111111,0.222222,0.222222,0.333333,0.111111,0.148148,0.133333,0.133333,0.004115,0.020282,0.032334,0.065278,0.135533,0.179012,0.103947
150,1.627300,0.349454,0.222222,0.444444,0.555556,0.555556,0.222222,0.259259,0.222222,0.200000,0.012662,0.045313,0.079587,0.129625,0.218582,0.361111,0.164131
200,1.029500,0.366128,0.111111,0.333333,0.333333,0.444444,0.111111,0.148148,0.200000,0.177778,0.004115,0.024714,0.057975,0.099466,0.177054,0.240741,0.156489
250,1.029500,0.401706,0.222222,0.333333,0.444444,0.444444,0.222222,0.222222,0.244444,0.200000,0.012662,0.037376,0.071248,0.115950,0.214017,0.305556,0.178424
300,0.603000,0.428423,0.222222,0.444444,0.444444,0.444444,0.222222,0.296296,0.266667,0.188889,0.012662,0.054470,0.079184,0.111835,0.213354,0.333333,0.165686
350,0.603000,0.515953,0.333333,0.333333,0.333333,0.777778,0.333333,0.222222,0.222222,0.211111,0.020599,0.037376,0.062090,0.288355,0.268694,0.385317,0.181830
400,0.497200,0.452074,0.333333,0.333333,0.333333,0.666667,0.333333,0.222222,0.222222,0.211111,0.020599,0.037376,0.062090,0.343911,0.285478,0.375441,0.197360
450,0.497200,0.469710,0.333333,0.333333,0.333333,0.444444,0.333333,0.259259,0.222222,0.200000,0.020599,0.045923,0.062090,0.129625,0.224893,0.344444,0.188016


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Corpus Chunks: 100%|██████████| 1/1 [00:22<00:00, 22.77s/it]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:17<00:00, 17.10s/it]


In [71]:
from sentence_transformers.util import cos_sim

def eval_retriever(test_dataset,model_kwargs=None):
    if model_kwargs is None:
        model = model_init()
    else:
        model = SentenceTransformer(model_kwargs['trained_model_path'])
    
    ev = get_ret_eval(test_dataset)

    metrics = ev(model)
    print(metrics)

    test_cases = [
        ("It is raining", "It is not raining", 1),  # Contradict - should be CLOSE
        ("It is raining", "The weather is wet", 0),  # Confirm - should be FAR
    ]

    for s1, s2, expected in test_cases:
        emb1 = model.encode(s1)
        emb2 = model.encode(s2)
        sim = cos_sim(emb1, emb2).item()
        print(f"'{s1}' vs '{s2}'")
        print(f"  Similarity: {sim:.3f} (Expected: {'HIGH' if expected==1 else 'LOW'})")
        print()

In [231]:
from sentence_transformers import util
import numpy as np
from numpy import hstack

class OurClassifier:
    def __init__(self, retriever, final_classifier, corpus):
        self.retriever = retriever
        self.final_classifier = final_classifier
        self.corpus = corpus
        self.embeded_corpus = self.retriever.encode(corpus, convert_to_tensor=True)
        self.k = min(len(corpus), 10)

    def predict(self, x):
        embedded_x = self.retriever.encode(x, convert_to_tensor=True)
        top_k = util.semantic_search(embedded_x, self.embeded_corpus, top_k=self.k, score_function=self.retriever.similarity)
        predictions = []
        vectorizer_premise = TfidfVectorizer()
        vectorizer_hypothesis = TfidfVectorizer()
        for query_id, query in enumerate(x):
            prediction = 0  # Default to 0 if no match found
            for res in top_k[query_id]:
                corpus_id = res['corpus_id']
                score = res['score']
                
                # Get premise embedding and move to CPU, convert to numpy
                premise_embedding = self.embeded_corpus[corpus_id].cpu().numpy()
                
                # Get current query embedding, move to CPU, convert to numpy
                query_embedding = embedded_x[query_id].cpu().numpy()
                
                # Stack the embeddings
                combined_features = hstack([query, self.corpus[corpus_id]])
                
                # Make prediction
                prediction = self.final_classifier.predict([combined_features])[0]
                
                if prediction == 1:
                    break
                train_data_preprocessed = train_data.copy()
                test_data_preprocessed = test_data.copy()

                train_data_vectorised = train_data.copy()

                X_premise= vectorizer_premise.fit_transform(query)
                X_hypothesis = vectorizer_hypothesis.fit_transform(self.corpus[corpus_id])
                train_data_vectorised = hstack([X_premise, X_hypothesis])

                Y_premise = vectorizer_premise.transform(test_data_preprocessed["premise"])
                Y_hypothesis = vectorizer_hypothesis.transform(test_data_preprocessed["hypothesis"])
                test_data_vectorised = hstack([Y_premise, Y_hypothesis])
            predictions.append(prediction)
        
        return np.array(predictions)

In [232]:
from sentence_transformers import SentenceTransformer

retr = SentenceTransformer(model_mnlr2_kwargs['trained_model_path'])
m = OurClassifier(retr, svm_model, test_dataset['premise'])

In [233]:
print("predicting")
predictions = m.predict(np.array(list(set(test_dataset['hypothesis']))))
print("predicted")
h = set(test_dataset['hypothesis'])
dlabels = dict(zip(h, [0]*len(h)))

for i in range(len(test_dataset)):
    if test_dataset[i]['label'] == 1:
        dlabels[test_dataset[i]['X_hypothesis']] = 1

labels = np.array([v for k,v in dlabels.items()])
print(labels)
report_dict = classification_report(labels, predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

predicting


ValueError: could not convert string to float: np.str_('Receiving Party may acquire information similar to Confidential Information from a third party.')

In [177]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

model = SentenceTransformer(model_mnlr2_kwargs['trained_model_path'])
em_sim_ev = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset['hypothesis'],
    sentences2=test_dataset['premise'],
    scores=test_dataset['label'],
    main_similarity=SimilarityFunction.COSINE,
    show_progress_bar=True,
)

results = em_sim_ev(model)
print(results)

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

{'pearson_cosine': 0.3273217698261732, 'spearman_cosine': 0.3149717892857355}


In [78]:
model = SentenceTransformer(model_mnlr2_kwargs['trained_model_path'])
ev = get_ret_eval(valid_dataset)
res = ev(model)
for k,v in res.items():
    print(k,v)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:39<00:00, 39.93s/it]

cosine_accuracy@1 0.3333333333333333
cosine_accuracy@3 0.3333333333333333
cosine_accuracy@5 0.3333333333333333
cosine_accuracy@10 0.6666666666666666
cosine_precision@1 0.3333333333333333
cosine_precision@3 0.2222222222222222
cosine_precision@5 0.2222222222222222
cosine_precision@10 0.2111111111111111
cosine_recall@1 0.020598742820965042
cosine_recall@3 0.03737620404287071
cosine_recall@5 0.06209017320128431
cosine_recall@10 0.3439108216885995
cosine_ndcg@10 0.285478223191996
cosine_mrr@10 0.37544091710758376
cosine_map@100 0.19736013904280394


In [79]:
for k,v in res.items():
    print(k,v)

cosine_accuracy@1 0.3333333333333333
cosine_accuracy@3 0.3333333333333333
cosine_accuracy@5 0.3333333333333333
cosine_accuracy@10 0.6666666666666666
cosine_precision@1 0.3333333333333333
cosine_precision@3 0.2222222222222222
cosine_precision@5 0.2222222222222222
cosine_precision@10 0.2111111111111111
cosine_recall@1 0.020598742820965042
cosine_recall@3 0.03737620404287071
cosine_recall@5 0.06209017320128431
cosine_recall@10 0.3439108216885995
cosine_ndcg@10 0.285478223191996
cosine_mrr@10 0.37544091710758376
cosine_map@100 0.19736013904280394


In [76]:
for k,v in res.items():
    print(k,v)

cosine_accuracy@1 0.1
cosine_accuracy@3 0.2
cosine_accuracy@5 0.5
cosine_accuracy@10 0.5
cosine_precision@1 0.1
cosine_precision@3 0.13333333333333333
cosine_precision@5 0.22000000000000003
cosine_precision@10 0.22000000000000003
cosine_recall@1 0.0011363636363636363
cosine_recall@3 0.006742424242424242
cosine_recall@5 0.02645959595959596
cosine_recall@10 0.053585858585858584
cosine_ndcg@10 0.19891295583084478
cosine_mrr@10 0.2033333333333333
cosine_map@100 0.14136472872345446
